# Agulhas region particles


This brief tutorial shows how to recreate the [animated gif](http://oceanparcels.org/images/globcurrent_fullyseeded.gif) showing particles in the Agulhas region south of Africa.


We start with importing the relevant modules


In [1]:
from parcels import (
    FieldSet,
    ParticleSet,
    JITParticle,
    AdvectionRK4,
    ErrorCode,
    download_example_dataset,
)
from datetime import timedelta
import numpy as np

Now load the Globcurrent fields from the `GlobCurrent_example_data` directory (note that unlike in the main Parcels tutorial we don't use a dictionary for the filenames here; as they are the same for all variables, we don't need to)


In [2]:
example_dataset_folder = download_example_dataset("GlobCurrent_example_data")
filenames = f"{example_dataset_folder}/20*.nc"
variables = {
    "U": "eastward_eulerian_current_velocity",
    "V": "northward_eulerian_current_velocity",
}
dimensions = {"lat": "lat", "lon": "lon", "time": "time"}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

Now create vectors of Longitude and Latitude starting locations on a regular mesh, and use these to initialise a `ParticleSet` object.


In [3]:
lons, lats = np.meshgrid(range(15, 35), range(-40, -30))
pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lons, lat=lats)

Now we want to advect the particles. However, the Globcurrent data that we loaded in is only for a limited, regional domain and particles might be able to leave this domain. We therefore need to tell Parcels that particles that leave the domain need to be deleted. We do that using a `Recovery Kernel`, which will be invoked when a particle encounters an `ErrorOutOfBounds` error:


In [4]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

Now we can advect the particles. Note that we do this inside a `for`-loop, so we can save a plot every six hours (which is the value of `runtime`). See the [plotting tutorial](https://docs.oceanparcels.org/en/latest/examples/tutorial_plotting.html) for more information on the `pset.show()` method.


In [5]:
for cnt in range(3):
    # First plot the particles
    pset.show(
        savefile="particles" + str(cnt).zfill(2), field="vector", land=True, vmax=2.0
    )

    # Then advect the particles for 6 hours
    pset.execute(
        AdvectionRK4,
        runtime=timedelta(hours=6),  # runtime controls the interval of the plots
        dt=timedelta(minutes=5),
        recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},  # the recovery kernel
    )

/Users/erik/Codes/ParcelsCode/parcels/plotting.py:219: RuntimeWarning: invalid value encountered in divide
  u = np.where(speed > 0., data[0]/speed, 0)
/Users/erik/Codes/ParcelsCode/parcels/plotting.py:220: RuntimeWarning: invalid value encountered in divide
  v = np.where(speed > 0., data[1]/speed, 0)
INFO: Plot saved to particles00
INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /var/folders/1n/500ln6w97859_nqq86vwpl000000gr/T/parcels-504/lib3f4edd19128b8634f9b0b0b87617d047_0.so
/Users/erik/Codes/ParcelsCode/parcels/plotting.py:219: RuntimeWarning: invalid value encountered in divide
  u = np.where(speed > 0., data[0]/speed, 0)
/Users/erik/Codes/ParcelsCode/parcels/plotting.py:220: RuntimeWarning: invalid value encountered in divide
  v = np.where(speed > 0., data[1]/speed, 0)
INFO: Plot saved to particles01
INFO: Plot saved to particles02


This now has created 3 plots. Note that the original animated gif contained 20 plots, but to keep running of this notebook fast we have reduced the number here. Of course, it is trivial to increase the number of plots by changing the value in the `range()` in the cell above.


As a final step, you can use [ImageMagick](http://www.imagemagick.org/script/index.php) or an online tool to stitch these individual plots together in an animated gif.

![gif](images/globcurrent_fullyseeded.gif)